# 영화 추천: ranking 단계

실제 추천 시스템은 종종 두 단계로 구성됩니다.

1. 검색 단계(Retrieval Phase)에서는 가능한 모든 후보 중에서 수백개의 초기 후보 집합을 선택합니다. 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보를 효율적으로 제거하는 것입니다. 검색 모델은 수백만 개의 후보를 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

2. 순위 단계(Ranking Phase)에서는 검색 모델의 출력을 가져와서 가능한 한 가장 좋은 추천 항목을 선택하도록 미세 조정합니다. 이 작업은 사용자가 관심을 가질 만한 항목 집합을 가능한 후보의 최종 목록으로 좁히는 것입니다.

이 notebook 은 두번째 단계인 순위 단계 모델을 구현합니다.  

다음을 수행합니다.

1. 데이터를 가져와 훈련 세트와 테스트 세트로 나눕니다.
2. 순위 모델을 구현합니다.
3. 피팅하고 평가합니다.


## Preparing the dataset

[retrieval](basic_retrieval) 과 동일한 데이터를 사용합니다. 이번에는 등급(explicit feedback)도 유지합니다. 이는 우리가 예측하려는 target 입니다.

이전과 마찬가지로 train/test를 80% 대 20%로 분할합니다.

## 모델 구현

### Architecture

랭킹 모델은 검색 모델처럼 효율성 제약에 직면하지 않으므로 아키텍처를 선택할 때 조금 더 자유로워집니다.

여러 스택의 dense layer로 구성된 모델은 task 순위 지정을 위한 비교적 일반적인 아키텍처입니다. 다음과 같이 구현할 수 있습니다.

이 모델은 user ID와 movie title을 가져와 예상 등급을 출력합니다.

### Loss and metrics

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 작업이 있습니다.

이 경우 손실 함수와 메트릭 계산을 함께 묶는 편리한 래퍼인 `Ranking` 작업 개체를 사용합니다.

평점을 예측하기 위해 `MeanSquaredError` Keras 손실과 함께 사용할 것입니다.

task 자체는 true 와 predicted 를 인수로 받아 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 빌딩 모델을 간소화하는 base 모델 클래스(`tfrs.models.Model`)를 노출합니다. feature를 제공하고 손실 값을 반환합니다.

그러면 base model이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

## Fitting and evaluating

훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

 모델을 훈련합니다.

RMSE 측정항목이 낮을수록 모델이 등급을 예측하는 경우가 더 정확해집니다.

## Testing the ranking model

이제 영화 세트에 대한 예측을 계산하여 ranking model을 테스트한 다음 예측을 기반으로 이러한 영화의 순위를 지정할 수 있습니다.
